# CSC-321: Data Mining and Machine Learning
# Matthew Caulfield
## Assignment 8: K-Nearest Neighbor

### Part 1: Implementation

For this assignment, I'm going to let you break down the implementation as you see fit. You're going to implement KNN. In brief, this means:

- calculating euclidean distance between a test instance, and a training instance
- iterating through all the training instances, and storing distances in a list
- returning the k nearest neighbors
- making a prediction by choosing the class that appears the most in the k nearest neighbors

To calculate euclidean distance between an instance x1 and an instance x2, we need to iterate through the features (excluding the class) of the two instances (for i features) and for each take the difference of (x1[i]) - (x2[i]), squaring that difference, and summing over all features. At the end, take the square root of the total. In other words:

$$distance=\sqrt{\sum_{i=1}^n (x1_{i} - x2_{i})^2}$$

I didn't really need to include this equation, but now you have an example of embedding a latex equation inside markdown. You're welcome.

I would strongly suggest you follow the implementation outline of previous algorithms in terms of the functions you use, but I'm leaving it up to you.

Below is the same contrived dataset you've used before. If your code works, you should be able to take an instance of this data, and compare it to all the others (including itself, where the distance SHOULD be 0). You should be able to select the k-nearest neighbors, and make a prediction based on the most frequently occuring class.

Make sure you create a knn function that takes a training set, a test set and a value for k.


In [1]:
# Contrived data set
import operator

dataset = [[3.393533211,2.331273381,0],
    [3.110073483,1.781539638,0],
    [1.343808831,3.368360954,0],
    [3.582294042,4.67917911,0],
    [2.280362439,2.866990263,0],
    [7.423436942,4.696522875,1],
    [5.745051997,3.533989803,1],
    [9.172168622,2.511101045,1],
    [7.792783481,3.424088941,1],
    [7.939820817,0.791637231,1]]

def euclidDistance(x1, x2):
    distance = 0
    for i in range(len(x1)-1):
        distance += (x1[i] - x2[i])**2
    totalDistance = distance**(0.5)
    return totalDistance

test = []
for j in dataset:
    test.append(euclidDistance(dataset[0],j))
    
print('distance test', test)

def kClosest(aList, k):
    kList = []
    copyList = aList[:]
    for i in range(k):
        currMin = min(copyList)
        #print(currMin)
        kList.append(currMin)
        #print(kList)
        #print(aList)
        copyList.pop(copyList.index(currMin))
    return kList

        
def knn(train, test, k):
    results = []
    for i in range(len(test)):
        testDistances = []
        for j in range(len(train)):
            testDistances.append(euclidDistance(test[i], train[j]))
        kClosestList = kClosest(testDistances, k)
        nearistNeighbors = [train[testDistances.index(i)] for i in kClosestList]
        nearistClasses = [i[-1] for i in nearistNeighbors]
        results.append(max(set(nearistClasses), key=nearistClasses.count))
    return results





distance test [0.0, 0.6185116050573538, 2.2971549072793094, 2.355481259443775, 1.2353710961872457, 4.672743225343651, 2.6412435314940947, 5.7814327983643325, 4.532951443185023, 4.799917756676257]


### Part 2: Working with real data

Apply the KNN algorithm above to the abalone data set. You can find more about it here: http://archive.ics.uci.edu/ml/datasets/Abalone

You will need to make the class value into an integer class. Run a 5-fold cross-validation, with k set as 5. Also run a classification baseline. Report on classification accuracy, and write up some results.


In [2]:
import statistics
import random
import csv
import math
from collections import Counter

def mean(listOfValues):
    total = 0
    for num in listOfValues:
        total += num
    return total/len(listOfValues)

def variance(listOfValues, meanValue):
    total = 0
    for num in listOfValues:
       total +=  (num - meanValue)**2/(len(listOfValues)-1)
    return total

def load_data(filename):
    csvTxt = csv.reader(open(filename))
    data = []
    for row in csvTxt:
        data.append(row)
    return data

def column2Float(dataset,column):
    for instance in dataset:
        instance[column] = float(instance[column])
    return dataset

def rmse_eval(actual, predicted):
    error = 0.0
    for i in range(len(actual)):
        error += (predicted[i] - actual[i])**2
    error = error/len(actual)
    error = error**0.5
    return error

def minmax(dataset):
    listMinMax = []
    for column in range(len(dataset[0])):
        columnData = [dataset[i][column] for i in range(len(dataset))]
        listMinMax.append([min(columnData), max(columnData)])
    return listMinMax

def normalize(dataset, minmax):
    for row in range(len(dataset)):
        for column in range(len(dataset[row])):
            dataset[row][column] = (dataset[row][column] - minmax[column][0]) / (minmax[column][1] - minmax[column][0])

def accuracy(actual, predicted):
    counter = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            counter += 1
    return counter*100/len(actual)

def zeroRC(train, test):
    trainY = [i[-1] for i in train]
    count = Counter(trainY)
    dataMode = count.most_common(1)[0][0]
    return [dataMode for i in test]

random.seed(1)

def cross_validation_data(dataset, folds):
    dataCopy = dataset[:]
    foldLen = len(dataCopy)//folds
    crossData = []
    for i in range(folds - 1):
        currFold = []
        for j in range(foldLen):
            currData = random.choice(dataCopy)
            currFold.append(currData)
            dataCopy.pop(dataCopy.index(currData))
        crossData.append(currFold)
    currFold = []
    for i in range(len(dataCopy)):
            currData = random.choice(dataCopy)
            currFold.append(currData)
            dataCopy.pop(dataCopy.index(currData))
    crossData.append(currFold)
    return crossData

def evaluate_algorithm(dataset, algorithm, folds, metric, *args):
    foldedData = cross_validation_data(dataset, folds)
    scores = []
    for i in range(len(foldedData)):
        copyFolded = foldedData[:]
        test_data = copyFolded.pop(i)
        test = [test_data[j][:-1] for j in range(len(test_data))]
        for j in test:
            j.append(None)
        train = []
        for fold in copyFolded:
            train += fold
        predicted = algorithm(train,test, *args)
        actual = [j[-1] for j in test_data]
        result = metric(actual,predicted)
        scores.append(result)
    return scores


filename = 'abalone.csv'
abaloneData = load_data(filename)
print('Number of Instances:', len(abaloneData), 'Number of Features:', len(abaloneData[0]))
for column in range(1,len(abaloneData[0])):
    column2Float(abaloneData, column)
    
def abaloneClass(data):
    for i in data: 
        if i[0] == 'M':
            i[0] = 1
        elif i[0] == 'F':
            i[0] = 0
        elif i[0] == 'I':
            i[0] = 2
            
abaloneClass(abaloneData)

abaloneCopy = abaloneData[:]
print('abaloneData first row', abaloneCopy[0])
folds = 5
k = 5
scores = evaluate_algorithm(abaloneCopy, knn, folds, accuracy,k)
zeroRCScores = evaluate_algorithm(abaloneCopy, zeroRC, folds, accuracy)
print('knn:', scores)
print('knn Min: %.3f' % min(scores), 'knn Max: %.3f' % max(scores), 'knn Mean: %.3f' % mean(scores))
print('zeroRC: ', zeroRCScores)
print('zeroRC Min: %.3f' % min(zeroRCScores), 'zeroRC Max: %.3f' % max(zeroRCScores), 'zeroRC Mean: %.3f' % mean(zeroRCScores))

Number of Instances: 4177 Number of Features: 9
abaloneData first row [1, 0.455, 0.365, 0.095, 0.514, 0.2245, 0.101, 0.15, 15.0]
knn: [24.910179640718564, 21.79640718562874, 23.592814371257486, 21.676646706586826, 23.416965352449225]
knn Min: 21.677 knn Max: 24.910 knn Mean: 23.079
zeroRC:  [15.568862275449101, 17.604790419161677, 15.688622754491018, 17.604790419161677, 16.009557945041816]
zeroRC Min: 15.569 zeroRC Max: 17.605 zeroRC Mean: 16.495


#### Write up results here
Knn worked better than zeroR for classification of the abalone dataset but not by much KNN mean was 23% while zeroR mean 2as 16.5%. From the results I would not reccomend using KNN on this dataset and we should try another classification algorithm instead. We also do not know from this which attributes were dominant in classification and if by removing some attributes we may have better results, because all we know are which datapoints are closest to the training datapoints. From this we may be able to tell that the data did not have distinct clusters based on age alone. 

### Part 3: KNN regression

We can also run KNN as a regression algorithm. In this case, instead of predicting the most common class in the k nearest neighbors, we can assign a predicted value that is the mean of the values in the k neighbors. 

Make this change to your algorithm (presumably by simply implementing a new predict function below, because you divided your code up sensibly in Part 1), and run the abalone data as a regression problem (convert the class data to a float, before running the algorithm). Use the same number of folds and the same k value as before. Also run a regression baseline and report RMSE values for both. Give me some explanation of the results, both standalone and in comparison to the classification results above.


In [3]:

def knnRegression(train, test, k):
    results = []
    for i in range(len(test)):
        testDistances = []
        for j in range(len(train)):
            testDistances.append(euclidDistance(test[i], train[j]))
        kClosestList = kClosest(testDistances, k)
        nearistNeighbors = [train[testDistances.index(i)] for i in kClosestList]
        nearistClasses = [i[-1] for i in nearistNeighbors]
        results.append(mean(nearistClasses))
    return results

def zeroRR(train, test):
    trainY = [i[1] for i in train]
    testY = [i[1] for i in test]

    trainYMean = mean(trainY)
    predictions = []
    for i in testY:
        predictions.append(trainYMean)
    return predictions

folds = 5
k = 5
scores = evaluate_algorithm(abaloneCopy, knnRegression, folds, rmse_eval,k)
zeroRRScores = evaluate_algorithm(abaloneCopy, zeroRR, folds, rmse_eval)
print('knnRegression:', scores)
print('knnRegression Min: %.3f' % min(scores), 'knnRegression Max: %.3f' % max(scores), 'knnRegression Mean: %.3f' % mean(scores))
print('zeroRR: ', zeroRRScores)
print('zeroRR Min: %.3f' % min(zeroRRScores), 'zeroRR Max: %.3f' % max(zeroRRScores), 'zeroRR Mean: %.3f' % mean(zeroRRScores))

knnRegression: [2.169566307830174, 2.272512129174563, 2.1622786827318077, 2.1966114350078354, 2.379431977435985]
knnRegression Min: 2.162 knnRegression Max: 2.379 knnRegression Mean: 2.236
zeroRR:  [9.893305049173978, 9.897647465191824, 9.884595083099303, 10.045356035694127, 10.010862489546504]
zeroRR Min: 9.885 zeroRR Max: 10.045 zeroRR Mean: 9.946


#### Write up results here

KNN regression worked much better then the base line of zeroR with an RMSE of 2.26 versus an RMSE of 10. Without knowing how RMSE relates to classification I cannot for sure say that KNN regression is better than KNN classification for this dataset however it outperformed its base line much more than KNN classification did. This may mean that the age of the abalone followed some linear pattern based on its attributes where clusters may overlap but older abalone had similar features and younger abalone had similar features. 